- 나연님폴더에서 sampro_1y.csv, sampro_1y_2.csv

In [23]:
import pandas as pd
import soynlp
from soynlp.noun import LRNounExtractor_v2
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

import warnings
warnings.filterwarnings('ignore')

In [24]:
def clean_sents_df(company):
    try:
        target = pd.read_csv(f'C:/sh/study/krx데이콘/krx_2022/sh/data/data/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'C:/sh/study/krx데이콘/krx_2022/sh/data/data/{company}_2year.csv',encoding='utf8')

    if company == 'sampro1' or company == 'sampro2':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open(r"C:\sh\study\krx데이콘\krx_2022\sh\data\data\corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()

    
def corpus_init():
    ktop30_company = pd.read_excel(r'C:\sh\study\krx데이콘\krx_2022\sh\data\data\KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro1')
    company_set.append('sampro2')
    f = open(r"C:\sh\study\krx데이콘\krx_2022\sh\data\data\corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus(r"C:\sh\study\krx데이콘\krx_2022\sh\data\data\corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [25]:
corpus_init()
tokenizer = return_tokenizer()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1871785 from 3181877 sents. mem=3.007 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=16231321, mem=6.057 Gb
[Noun Extractor] batch prediction was completed for 460511 words
[Noun Extractor] checked compounds. discovered 464165 compounds
[Noun Extractor] postprocessing detaching_features : 411493 -> 313632
[Noun Extractor] postprocessing ignore_features : 313632 -> 313101
[Noun Extractor] postprocessing ignore_NJ : 313101 -> 307570
[Noun Extractor] 307570 nouns (464165 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=6.855 Gb                    
[Noun Extractor] 73.32 % eojeols are covered


In [28]:
ktop30_company = pd.read_excel(r'C:\sh\study\krx데이콘\krx_2022\sh\data\data\KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
company_set = list(ktop30_company)
company_set.append('sampro1')
company_set.append('sampro2')
company_set

['삼성전자',
 'NAVER',
 '삼성SDI',
 'LG화학',
 '카카오',
 '유한양행',
 'SK텔레콤',
 'POSCO홀딩스',
 '현대모비스',
 'SK이노베이션',
 '삼성화재',
 '롯데케미칼',
 '현대차',
 '셀트리온',
 '삼성전기',
 '아모레퍼시픽',
 '삼성물산',
 '이마트',
 'CJ ENM',
 'SK하이닉스',
 '한국조선해양',
 'LG전자',
 '기아',
 '넷마블',
 '삼성생명',
 'KB금융',
 '현대건설',
 '신한지주',
 'LG디스플레이',
 '미래에셋증권',
 'sampro1',
 'sampro2']

In [27]:
print(company_set)

['삼성전자', 'NAVER', '삼성SDI', 'LG화학', '카카오', '유한양행', 'SK텔레콤', 'POSCO홀딩스', '현대모비스', 'SK이노베이션', '삼성화재', '롯데케미칼', '현대차', '셀트리온', '삼성전기', '아모레퍼시픽', '삼성물산', '이마트', 'CJ ENM', 'SK하이닉스', '한국조선해양', 'LG전자', '기아', '넷마블', '삼성생명', 'KB금융', '현대건설', '신한지주', 'LG디스플레이', '미래에셋증권', 'sampro1', 'sampro2']


In [14]:
target_df = clean_sents_df("삼성전자")
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글
0,2022-07-11 23:42,요즘 한전 게시판 조용하네!ㅋ,2,0,0,요즘 한전 게시판 조용하네,"[요즘, 한전, 게시판, 조용, 하네]"
1,2022-07-11 23:42,1% 올려야 함,4,0,0,올려야 함,"[올려야, 함]"
2,2022-07-11 23:42,=자신들잘못된죄들=모르는=내로남불들=척결...,3,0,0,자신들잘못된죄들 모르는 내로남불들 척결,"[자신들, 잘못된죄들, 모르는, 내로남불, 들, 척결]"
3,2022-07-11 23:42,난 내일 가요,9,0,0,난 내일 가요,"[난, 내일, 가요]"
4,2022-07-11 23:42,실적으로 이만한 실적내는 기업 있으면 나...,6,0,0,실적으로 이만한 실적내는 기업 있으면 나,"[실적, 으로, 이만, 한, 실적, 내는, 기업, 있으면, 나]"
...,...,...,...,...,...,...,...
952447,2020-06-01 00:09,내8자,707,1,3,내 자,"[내, 자]"
952448,2020-06-01 00:08,하락낙폭저지 의 길,682,2,0,하락낙폭저지 의 길,"[하락낙폭저지, 의, 길]"
952449,2020-06-01 00:07,주주님들 오랜만에 오르니까 좋네요 ㅎㅎㅎ,697,1,0,주주님들 오랜만에 오르니까 좋네요,"[주주님, 들, 오랜만, 에, 오르니까, 좋네요]"
952450,2020-06-01 00:03,코덱스200선물인버스 곱배기..,725,0,0,코덱스 선물인버스 곱배기,"[코덱스, 선물, 인버스, 곱배기]"


In [15]:
type(target_df)

pandas.core.frame.DataFrame

In [19]:
company = "삼성전자_전처리.csv"
target_df.to_csv(r"C:\sh\study\krx데이콘\krx_2022\sh\data\preprocessed" + "/" + company)

In [29]:
for company in company_set:
    target_df = clean_sents_df(company)
    target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
    target_df.to_csv(r"C:\sh\study\krx데이콘\krx_2022\sh\data\preprocessed" + "/" + company + ".csv")

# 레이블링 & 랜덤추출

---

In [5]:
def labeling(target_df):
    
    f = open("./src/neg_pol_word.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    fear_words_set = {word.strip('\n') for word in words}

    f = open("./src/pos_pol_word.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    greed_words_set = {word.strip('\n') for word in words}
    
    label_score = []
    for token_list in target_df['토큰화 댓글']:
        sent_score = 0
        for token in token_list:
            if token in fear_words_set:
                sent_score -= 1
            elif token in greed_words_set:
                sent_score += 1

        if sent_score < 0:
            label_score.append(-1)
        elif sent_score > 0:
            label_score.append(1)
        else:
            label_score.append(0)
            
    target_df['label'] = label_score
    
    return target_df


In [8]:
def setting_train_data():
    company = list(pd.read_excel('./KODEX_KTOP_30_20220715.xls',header=2)['종목명'][1:])
    dominance = list(pd.read_excel('./KODEX_KTOP_30_20220715.xls',header=2)['비중(%)'][1:])

    train_data = pd.DataFrame(columns=['날짜','댓글','조회수','좋아요','싫어요','정제된 댓글','토큰화 댓글','label'])
    for idx in range(30):
        target_df = clean_sents_df(company[idx])
        target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
        
        label_df = labeling(target_df)
        
        label_df = label_df[label_df['label'] != 0]
        label_df = label_df.sample(int(30000*dominance[idx])+1,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
        train_data = train_data.append(label_df)
    
    return train_data

In [9]:
train_data = setting_train_data()
train_data.reset_index(inplace=True)
train_data.drop('index',axis=1,inplace=True)
train_data

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글,label
0,2020-08-19 19:49,오늘도 좋은 하루되세용~~!,257,0,0,오늘도 좋은 하루되세용,"[오늘, 도, 좋은, 하루, 되세용]",1
1,2021-04-09 10:29,빨개이들 선거 져서 분노표출,331,3,1,빨개이들 선거 져서 분노표출,"[빨개이들, 선거, 져서, 분노표출]",-1
2,2020-12-18 16:20,애써 들여다보지 말라. 부정을 보았다면.,315,1,0,애써 들여다보지 말라 부정을 보았다면,"[애써, 들여다보지, 말라, 부정, 을, 보았다, 면]",-1
3,2021-04-19 10:45,좋은 뉴스는 많은데...,493,0,0,좋은 뉴스는 많은데,"[좋은, 뉴스, 는, 많은데]",1
4,2021-01-04 11:47,짜증나네 84300 천주 사자마자물리네,576,4,0,짜증나네 천주 사자마자물리네,"[짜증, 나네, 천주, 사자, 마자물리네]",-1
...,...,...,...,...,...,...,...,...
30008,2021-10-27 11:05,답글:개행주의 저주는,239,1,2,개행주의 저주는,"[개행주, 의, 저주, 는]",-1
30009,2021-11-16 12:27,회장님 비난은 자재하셨음 하내여~,307,1,4,회장님 비난은 자재하셨음 하내여,"[회장님, 비난, 은, 자재, 하셨음, 하내여]",-1
30010,2020-12-10 14:37,퉤~~~더러운 주식...,304,15,2,퉤 더러운 주식,"[퉤, 더러운, 주식]",-1
30011,2020-08-06 11:56,감사합니다,338,4,1,감사합니다,"[감사, 합니다]",1


---

In [10]:
# 삼프로
target_df = clean_sents_df('sampro')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
label_df = labeling(target_df)

label_df = label_df[label_df['label'] != 0]
label_df = label_df.sample(10000,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
train_data = train_data.append(label_df)

In [11]:
train_data = train_data.loc[:,['정제된 댓글','label']].reset_index().drop('index',axis=1)

In [12]:
train_data

,정제된 댓글,label
0,오늘도 좋은 하루되세용,1
1,빨개이들 선거 져서 분노표출,-1
2,애써 들여다보지 말라 부정을 보았다면,-1
3,좋은 뉴스는 많은데,1
4,짜증나네 천주 사자마자물리네,-1
...,...,...
40008,박부장님 좋은 말씀 감사합니다,1
40009,니가 잘 나갈거라 한거 죽수고 있다 이사되니 좋으냐 좀 겸손한 모습 보여라,1
40010,이정봉아티스트님 좋은 음악 들려주셔서 감사합니다,1
40011,일상이 삼프로와 함께네요 늘 감사합니다,1


In [14]:
for i in range(len(train_data)):
    if train_data['label'][i] == -1:
        train_data['label'][i] = 0

train_data

,정제된 댓글,label
0,오늘도 좋은 하루되세용,1
1,빨개이들 선거 져서 분노표출,0
2,애써 들여다보지 말라 부정을 보았다면,0
3,좋은 뉴스는 많은데,1
4,짜증나네 천주 사자마자물리네,0
...,...,...
40008,박부장님 좋은 말씀 감사합니다,1
40009,니가 잘 나갈거라 한거 죽수고 있다 이사되니 좋으냐 좀 겸손한 모습 보여라,1
40010,이정봉아티스트님 좋은 음악 들려주셔서 감사합니다,1
40011,일상이 삼프로와 함께네요 늘 감사합니다,1


In [15]:
train_data.to_csv('./train_data.csv')